<a href="https://colab.research.google.com/github/mayank-soni/text_summary/blob/transformer_train/notebooks/transformer_train_mayank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install requirements

In [4]:
! python --version

Python 3.8.15


In [ ]:
! pip install transformers datasets
! pip install rouge-score nltk
# ! pip install huggingface_hub

#Set parameters

In [2]:
model_checkpoint = 'sshleifer/distilbart-cnn-12-6'
dataset_name = 'xsum'
metric_name = 'rouge'

# Loading data

In [3]:
import transformers

In [9]:
from datasets import load_from_disk
raw_datasets_t = load_from_disk('train_data')
raw_datasets_v = load_from_disk('validation_data')

In [10]:
print (raw_datasets_t)
print(raw_datasets_v)

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 1020
})
Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 1232
})


In [15]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5, random_seed=36):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    random.seed(random_seed)
    picks = random.sample(range(len(dataset)), num_examples)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
      if isinstance(typ, datasets.ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    return df

In [17]:
data_t = show_random_elements(raw_datasets_t)
# data.head()

,article,highlights,id
0,"Siem de Jong played 45 minutes for Newcastle Under 21s on his comeback from a collapsed lung. The 26-year-old made way at the break after coming through the first half of United's 2-0 defeat to Derby County at St James' Park. De Jong has made just one Premier League start since his £6million arrival from Ajax last summer. Newcastle midfielder Siem de Jong played 45 minutes for Peter Beardsley's Under 21 side on Wednesday . Farrend Rawson (third from right) is congratulated by his team-mates after scoring at St James' Park . He tore his thigh muscle in September and was days away from returning in February when, for the second time in two years, he suffered a collapsed lung and underwent surgery. De Jong was expected to miss the remainder of the season but now has an outside chance of featuring in the final six matches. However, on the evidence of a subdued half against the Rams, Sunday's visit of Spurs will almost certainly come too soon. Newcastle striker Adam Armstrong (right) is pulled back by Derby's Max Lowe . Armstrong (left) heads a cross towards Derby's goal but his effort flies wide of the post . Meanwhile, forgotten man Facundo Ferreyra replaced De Jong at the break. The 24-year-old Argentinian striker arrived on a season-long loan from Shakhtar Donetsk last summer but is yet to make his senior debut having been plagued by a string of injuries. Derby's victory arrived courtesy of second-half goals from Alefe Santos and Farrend Rawson.",Siem de Jong has made just one Premier League start this season .\nThe 26-year-old has been plagued by injuries since joining from Ajax .\nHe played 45 minutes as Newcastle lost 2-0 at home to Derby County .\nLoanee Facundo Ferreyra replaced De Jong at half-time .\nAlefe Santos and Farrend Rawson on target for the Rams at St James' Park .,f7b25ae2d51010ec62051aa98b16cd296e30ea8e
1,"Reigning champion Novak Djokovic dug deep to avoid a shock exit at the hands of Alexandr Dolgopolov before powering into the quarter-finals of the Miami Open. The unseeded Dolgopolov - ranked 65th in the world, 64 places behind Djokovic - surprised the Serbian by winning a first-set tie-break and building a quick 4-2 lead in the second set. But Djokovic steeled himself and rallied to force a decider, which he wrapped up without much resistance from his deflated Ukrainian opponent. Novak Djokovic celebrates after securing a quarter-final place at the Miami Open . Novak Djokovic came back from a set down to reach the quarter-finals of the Miami Open on Tuesday . Alexandr Dolgopolov had both of his soles bandaged and won only three points in the final set . Djokovic claimed a succession of long exchanges in the second set to even the match and go on to win . The 6-7 (3/7) 7-5 6-0 triumph pits the world number one against David Ferrer in the last eight after the Spaniard saw off Gilles Simon 7-6 (7/5) 6-0 in a little over an hour and a half. Also through to the quarters are British number one Andy Murray and fourth seed Kei Nishikori. The Scot racked up a 500th career win by edging out Kevin Anderson 6-4 3-6 6-3 to book a meeting with Austrian Dominic Thiem, a 7-6 (7/5) 4-6 7-5 victor against Adrian Mannarino. Nishikori of Japan made light work of David Goffin and his 6-1 6-2 sees him progress to a match-up against big-serving American John Isner, who rallied to upset fifth seed Milos Raonic 6-7 (3/7) 7-6 (8/6) 7-6 (7/5) in a match which last two and three-quarter hours. David Ferrer celebrates after defeating Gilles Simon to book his place in the last eight . Simon is grounded after a fall during his defeat to Ferrer, who faces Novak Djokovic next . Andy Murray was given a celebratory cake to mark 500 wins on the ATP circuit after beating Kevin Anderson . Tomas Berdych also made it through to the quarter-finals by beating Gael Monfils . Kei Nishikori is also through to the last eight, defeating Serbia's Viktor Troicki . Czech challenger Tomas Berdych reached his sixth quarter-final of 

In [18]:
data_v = show_random_elements(raw_datasets_v)

,article,highlights,id
0,"Illinois Rep. Aaron Schock billed taxpayers for more than $10,000 on private flights last fall, including a trip on a private jet to a Chicago Bears football game. The flights last November don't include more than a dozen other trips on donors' planes that Schock has paid for out of his House office expenses or campaign funds since joining Congress in 2009. Schock, a 33-year-old Republican representing Peoria, had already been facing scrutiny for using congressional funds to redecorate his Capitol Hill office in the style of the TV show 'Downton Abbey.' He repaid those charges last month out of his own pocket; his office had said it's reviewing other expenses. Rep. Aaron Schockis accused of billing taxpayers for more than $10,000 on private flights last fall that included a trip to a Chicago Bears football game . Schock and his entourage flew in an Eclipse 500 aircraft like this one . The Chicago Sun-Times reported Sunday night that Schock flew from Peoria, Illinois to Chicago to attend a Bears game against the Minnesota Vikings on November 16, 2014 . Overall, according to the paper, he used $20,855 in taxpayer money for the Chicago trip and another to New York in September, where his political action committee spent another $3,000 for tickets to the Global Citizen Festival concert. This week Schock reportedly hired a pair of lawyers to audit his political and legislative operations. Spokesmen for Schock declined to comment to the Sun-Times and the Chicago Tribune, and did not respond to a request from Daily Mail Online. In a statement last week they said a legal team was already poring over 'compliance procedures in his official office, campaign and leadership PAC to determine whether they can be improved.' The football game was part of a longer trip on a chartered six-seat business jet that took off from a civil aviation airport in Manassas, Virginia en route to Peoria on a Friday. The Bears defeated the Vikings but the game may ultimately defeat a congressman . DOWNTON OFFICE: Schock spent $40,000 in taxpayer funds redecorating his Capitol Hill office, but had to pay the money back . The Sunday trip was a sideline before the aircraft returned to Washington, D.C.'s Ronald Reagan National Airport a day later. All the seats were filled, according to pilot Keith Siilats, who told the Sun-Times that Schock's district director Dayne LaHood and three others were aboard. He also said he accompanied the congressman and his group to the football game. He was paid $10,802 for 'commercial transportation' in November, the last month for which numbers are available. 'That whole weekend was paid by the government,' he said. Schock 'called me on Saturday and said ""what do you think about’ a Chicago trip?""' Siilats recalled for the paper. Chicago Bears single-game tickets range in cost from $106 to $430. It's not clear who paid for them.","Illinois Republican used his House office expense budget for more than $10,000 in private flights including the November NFL jaunt .\nTook his district director, his pilot and three others to see the Bears play the Minnesota Vikings in November .\nHe has had to pay back taxpayers for $40,000 in office renovations styled after 'Downton Abbey'",fec80230c01b5c71fc7598c3e035be33cb787331
1,"Arsenal legend Robert Pires' son is following in his father's footsteps after training with the club's academy. Theo Pires, seven, completed his first training session at the club's Hale End youth complex in Walthamstow this week. Pires spent six years at Arsenal, helping Arsene Wenger's side dominate English football winning two Premier League titles and three FA Cups during his stay in north London. Robert Pires' son Theo, 7, pictured with his father and Alexis Sanchez, has trained with the Arsenal academy . The youngster was spotted at the Emirates Stadium earlier this year watching Arsenal beat Stoke City . Theo completed his first session at the Hale End academy in Walthamstow this week . An

#Load metric

In [19]:
from datasets import load_metric
metric = load_metric(metric_name)

<ipython-input-19-a88b822e44cf>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric(metric_name)


# Pre-process data

In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [21]:
prefix_models = ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]
if model_checkpoint in prefix_models:
  prefix = "summarize: "
else:
  prefix = ''

In [22]:
def preprocess_function(data):
  inputs = [prefix + doc for doc in data["article"]]
  tokenized_data = tokenizer(text=inputs, truncation=True, text_target=data['highlights'])
  return tokenized_data 

In [23]:
preprocess_function(raw_datasets_t[:2])
preprocess_function(raw_datasets_v[:2])

{'input_ids': [[0, 1640, 16256, 43, 3399, 22965, 585, 307, 14, 24, 34, 4639, 63, 5436, 9, 1393, 12255, 6926, 611, 6, 442, 123, 4973, 7, 671, 7, 5, 2414, 1320, 480, 13176, 22, 5087, 27495, 9505, 72, 6926, 611, 21, 3456, 71, 10, 8951, 2366, 461, 303, 14, 37, 1153, 2021, 1897, 1476, 136, 39, 320, 6096, 6, 12372, 925, 4473, 3937, 4, 264, 1238, 5, 14177, 1393, 9, 16004, 69, 30, 5, 14599, 8, 26963, 69, 471, 136, 10, 2204, 11, 39, 4243, 184, 23, 8951, 18, 21860, 1016, 13243, 11, 772, 4, 34192, 6, 5, 10591, 4482, 968, 2234, 9826, 39, 27495, 5436, 8, 685, 258, 498, 4, 280, 2425, 37, 2039, 5, 191, 12, 12211, 19627, 1764, 25, 157, 25, 80, 7757, 13780, 968, 4694, 4, 125, 37, 197, 28, 441, 7, 3511, 149, 5, 1136, 6, 8, 10591, 161, 14, 24, 40, 27673, 63, 7404, 13, 123, 7, 3511, 11, 70, 2836, 1061, 4, 20, 403, 136, 6926, 611, 362, 10, 1233, 1004, 94, 186, 6, 77, 5, 8951, 641, 9, 1659, 585, 14, 1103, 74, 45, 28, 1658, 136, 123, 4, 22, 1620, 38, 33, 26, 31, 5, 1786, 6, 38, 222, 45, 6225, 1897, 2134, 60,

In [24]:
tokenized_datasets_t = raw_datasets_t.map(preprocess_function, batched=True)
tokenized_datasets_v = raw_datasets_v.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

#Fine-tuning

TODO -> check if the unused weights are problematic

In [27]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [25]:
#batch_size = 8
batch_size = 1
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [29]:
#tokenized_datasets["train"]
print(tokenized_datasets_t)
print(tokenized_datasets_v)

Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1020
})
Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1232
})


TODO: Understand why validation set is processed twice, once for validation and once for generation

In [30]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets_t,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets_v,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets_v,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=generation_data_collator
)


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [31]:
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Consider adding a KerasMetricCallback:
is a callback for computing advanced metrics. There are a number of common metrics in NLP like ROUGE which are hard to fit into your compiled training loop because they depend on decoding predictions and labels back to strings with the tokenizer, and calling arbitrary Python functions to compute the metric. The KerasMetricCallback will wrap a metric function, outputting metrics as training progresses.

In [32]:
import numpy as np
import nltk

def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [33]:
import tensorflow as tf
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")
    model.fit(train_dataset, validation_data=validation_dataset, epochs=10) 
    #callbacks=callbacks)
   # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
      logical_gpus = tf.config.list_logical_devices('GPU')
      print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Epoch 1/10
1020/1020 [==============================] - 494s 447ms/step - loss: 1.6446 - val_loss: 1.5189
Epoch 2/10
1020/1020 [==============================] - 449s 440ms/step - loss: 1.0594 - val_loss: 1.5863
Epoch 3/10
1020/1020 [==============================] - 448s 439ms/step - loss: 0.7075 - val_loss: 1.7747
Epoch 4/10
1020/1020 [==============================] - 447s 438ms/step - loss: 0.4762 - val_loss: 2.0113
Epoch 5/10
1020/1020 [==============================] - 448s 439ms/step - loss: 0.3244 - val_loss: 2.2670
Epoch 6/10
1020/1020 [==============================] - 447s 438ms/step - loss: 0.2405 - val_loss: 2.4403
Epoch 7/10
1020/1020 [==============================] - 450s 441ms/step - loss: 0.1948 - val_loss: 2.4979
Epoch 8/10
1020/1020 [==============================] - 447s 439ms/step - loss: 0.1570 - val_loss: 2.6723
Epoch 9/10
1020/1020 [==============================] - 448s 439ms/step - loss: 0.1343 - val_loss: 2.7352
Epoch 10/10
1020/1020 [=======================

In [ ]:
'''
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./summarization_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True
)

callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]

model.fit(
    train_dataset, validation_data=validation_dataset, epochs=1) 
    #callbacks=callbacks) 
'''

In [46]:
document = raw_datasets_v[1]['article']

In [52]:
print(document)

(CNN)Former Manchester United star Danny Welbeck returned to haunt his old side to send holders Arsenal into the semifinals of the FA Cup with a 2-1 victory at Old Trafford Monday. The England striker was surplus to requirements at Louis van Gaal's United, but was quick to capitalize on a mistake by Antonio Valencia to grab the winner just after the hour mark. A disappointing night for United was compounded by a late red card for Argentina international Angel Di Maria, who was given his marching orders after manhandling referee Michael Oliver. Di Maria was initially booked for simulation and took his protests too far by grabbing the official, which led to his instant dismissal. With 10 men, United was never able to exert any real pressure on Arsenal and it took two fine saves David De Gea to deny further goals from Santi Cazorla and Alexis Sanchez for the away side. Victory for Arsenal booked a return to Wembley for a last four clash with either Reading or Bradford City, who are replay

In [47]:
# document = 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we\'re neglected or forgotten," she said.\n"That may not be true but it is perhaps my perspective over the last few days.\n"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"\nMeanwhile, a flood alert remains in place across the Borders because of the constant rain.\nPeebles was badly hit by problems, sparking calls to introduce more defences in the area.\nScottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.\nThe Labour Party\'s deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.\nHe said it was important to get the flood protection plan right but backed calls to speed up the process.\n"I was quite taken aback by the amount of damage that has been done," he said.\n"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."\nHe said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.\nHave you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.'
if 't5' in model_checkpoint: 
    document = "summarize: " + document
tokenized = tokenizer([document], return_tensors='np')
out = model.generate(**tokenized, max_length=128)

In [48]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

</s><s>Arsenal beat Manchester United 2-1 in the FA Cup on Monday night.
Danny Welbeck scored the only goal as United were held to a goalless draw.
Arsenal will face Reading or Bradford City in the last four.
Louis van Gaal's side will now be without a trophy in his first season.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [49]:
model_original = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [50]:
out_original = model_original.generate(**tokenized, max_length=128)


In [51]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out_original[0]))

</s><s> Arsenal beat Manchester United 2-1 in FA Cup semifinal at Old Trafford. Danny Welbeck scores winner to send holders Arsenal into last four. Angel Di Maria sent off for Man Utd after manhandling referee Michael Oliver. Manchester United without trophy in first season in charge of Louis van Gaal.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [38]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)